## 資料預處理
:label:`sec_pandas`

為了能用深度學習來解決現實世界的問題，我們經常從預處理原始資料開始，
而不是從那些準備好的張量格式資料開始。
在Python中常用的資料分析工具中，我們通常使用`pandas`軟體套件。
像龐大的Python生態系統中的許多其他擴充套件一樣，`pandas`可以與張量相容。
本節我們將簡要介紹使用`pandas`預處理原始資料，並將原始資料轉換為張量格式的步驟。
後面的章節將介紹更多的資料預處理技術。

## 讀取資料集

舉一個例子，我們首先(**建立一個人工資料集，並儲存在CSV（逗號分隔值）檔案**)
`../data/house_tiny.csv`中。
以其他格式儲存的資料也可以通過類似的方式進行處理。
下面我們將資料集按行寫入CSV檔案中。


In [2]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一個數據樣本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要[**從建立的CSV檔案中載入原始資料集**]，我們匯入`pandas`套件並呼叫`read_csv`函式。該資料集有四列三行。其中每行描述了房間數量（"NumRooms"）、巷子類型（"Alley"）和房屋價格（"Price"）。


In [3]:
# 如果沒有安裝pandas，只需取消對以下行的註釋來安裝pandas
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 處理缺失值

注意，"NaN"項代表缺失值。
[**為了處理缺失的數據，典型的方法包括*插值法*和*刪除法*，**]
其中插值法用一個替代值彌補缺失值，而刪除法則直接忽略缺失值。
在(**這裡，我們將考慮插值法**)。
 
通過位置索引`iloc`，我們將`data`分成`inputs`和`outputs`，
其中前者為`data`的前兩列，而後者為`data`的最後一列。
對於`inputs`中缺少的數值，我們用同一列的均值替換"NaN"項。


In [4]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
# 對數值列使用均值填充
numeric_cols = ['NumRooms']
inputs[numeric_cols] = inputs[numeric_cols].fillna(inputs[numeric_cols].mean())

# 對類別列使用特定值填充（例如 'NA' 或最常見的值）
categorical_cols = ['Alley']
inputs[categorical_cols] = inputs[categorical_cols].fillna('NA')

print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0    NA
2       4.0    NA
3       3.0    NA


[**對於`inputs`中的類別值或離散值，我們將"NaN"視為一個類別。**]
由於"巷子類型"（"Alley"）列只接受兩種類型的類別值"Pave"和"NaN"，
`pandas`可以自動將此列轉換為兩列"Alley_Pave"和"Alley_nan"。
巷子類型為"Pave"的行會將"Alley_Pave"的值設置為1，"Alley_nan"的值設置為0。
缺少巷子類型的行會將"Alley_Pave"和"Alley_nan"分別設置為0和1。


In [5]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_NA  Alley_Pave  Alley_nan
0       3.0     False        True      False
1       2.0      True       False      False
2       4.0      True       False      False
3       3.0      True       False      False


## 轉換為張量格式
 
[**現在`inputs`和`outputs`中的所有條目都是數值類型，它們可以轉換為張量格式。**]
當數據採用張量格式後，可以通過在 :numref:`sec_ndarray`中引入的那些張量函數來進一步操作。


In [6]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 0., 1., 0.],
         [2., 1., 0., 0.],
         [4., 1., 0., 0.],
         [3., 1., 0., 0.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## 小結

* `pandas`軟體包是Python中常用的數據分析工具中，`pandas`可以與張量相容。
* 用`pandas`處理缺失的數據時，我們可根據情況選擇用插值法和刪除法。

## 練習

創建包含更多行和列的原始數據集。
 
1. 刪除缺失值最多的列。
2. 將預處理後的數據集轉換為張量格式。


[Discussions](https://discuss.d2l.ai/t/1750)


In [7]:
#創建包含更多行和列的原始數據集。
 
#1. 刪除缺失值最多的列。
import pandas as pd

# Creating the dataset as described in the notebook
data = pd.DataFrame({
    "NumRooms": [None, 2, 4, None],
    "Alley": ["Pave", None, None, None],
    "Price": [127500, 106000, 178100, 140000]
})

# Identify the column with the most missing values and drop it
col_with_max_missing = data.isnull().sum().idxmax()
data_dropped = data.drop(columns=[col_with_max_missing])

# Show the dataset after dropping the column
col_with_max_missing, data_dropped


('Alley',
    NumRooms   Price
 0       NaN  127500
 1       2.0  106000
 2       4.0  178100
 3       NaN  140000)

In [8]:
# 2. 將預處理後的數據集轉換為張量格式。
# 填補剩餘的缺失值
data_dropped['NumRooms'].fillna(data_dropped['NumRooms'].mean(), inplace=True)

# 轉換為 PyTorch 張量
data_tensor = torch.tensor(data_dropped.values, dtype=torch.float32)

print(data_tensor)

tensor([[3.0000e+00, 1.2750e+05],
        [2.0000e+00, 1.0600e+05],
        [4.0000e+00, 1.7810e+05],
        [3.0000e+00, 1.4000e+05]])


C:\Users\marklai\AppData\Local\Temp\ipykernel_11280\2863181916.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_dropped['NumRooms'].fillna(data_dropped['NumRooms'].mean(), inplace=True)
